# Aula: operações com dataframes

Nessa aula trataremos de operações com objetos Pandas - Series e Dataframes, considerando basicamente duas fontes:

1. o livro Python para Análise de Dados, do Wes McKinney
2. o [guia](https://pandas.pydata.org/docs/user_guide/index.html) disponível na documentação do Pandas (para mim o melhor material sobre o tema)

In [1]:
import numpy as np
import pandas as pd

## Indexação Hierarquica - MultiIndex

Antes de começarmos a trabalhar alguns métodos para operações com os objetos Pandas, nós teremos contato com um recurso da biblioteca bastante importante: a Indexação Hierarquica, realizada pelo objeto Multiindex.

**Para que serve indexação hierarquica?**

Basicamente para nos ajudar a representar conjuntos de dados de dimensões maiores em dimensões menores. A dimensão de um conjunto de dados é o número de linhas e colunas e quanto maior essa dimensão, mais complicada a análise pode ser.

O Pandas ofere diversas opções de criação de multiindex, sendo a mais básica dela simplesmente fazer uma lista de listas para o parâmetro `index` do métodos Series ou dataframe. 

Vejamos o exemplo com uma series

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.451539
   2   -0.403930
   3    0.192704
b  1    1.207659
   3   -0.428835
c  1    0.489284
   2   -0.051864
d  2    0.179684
   3   -1.300690
dtype: float64

Notemos que, nos índices das linhas, temos um nível a mais de índices! Tanto que se formos verificar a lista de índices, agora, obtemos

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

De fato, temos 3 entradas referentes ao `a`, 2 ao `b`, ao `c` e ao `d`. 

Para dataframes, podemos ter multiplos indexação hierraquica tanto nas linhas quanto nas colunas.

In [4]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], 
                            [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

que, comparando com a experiência que muitas vezes temos no excel, por exemplo, equivale a

![](excel.png)

Além dessa criação direta, podemos utilizar o objeto `MultiIndex` diretamente. Para isso, a biblioteca Pandas traz vários métodos, que podem ser vistos em detalhes no [User guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

Para exemplificar, consideremos dois casos:

1. a criação a partir de listas/arrays, usando `MultiIndex.from_arrays()`:

In [5]:
lista = [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 3, 1, 2, 2, 3]]
lista

[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 3, 1, 2, 2, 3]]

In [6]:
indice = pd.MultiIndex.from_arrays(lista)
indice

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [7]:
pd.Series(np.random.randn(9),index=indice)

a  1   -0.057659
   2    0.435930
   3   -0.362912
b  1   -0.018591
   3    0.084079
c  1   -1.026358
   2   -0.371486
d  2   -0.624126
   3    0.533197
dtype: float64

2. a criação a partir de um dataframe sem indexação hierarquica, usando `MultiIndex.to_frame()`:

In [8]:
df = pd.DataFrame([["bar", "one"], ["bar", "two"], ["foo", "one"], ["foo", "two"]],
                  columns=["first", "second"])
df

,first,second
0,bar,one
1,bar,two
2,foo,one
3,foo,two


In [10]:
pd.MultiIndex.from_frame(df)

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('foo', 'one'),
            ('foo', 'two')],
           names=['first', 'second'])

In [11]:
df_ = pd.DataFrame([["bar", "one"], ["bar", "two"], ["foo", "one"], ["foo", "two"]],
                  columns=["first", "second"],
                  index=pd.MultiIndex.from_frame(df))
df_

first second
first second             
bar   one      bar    one
      two      bar    two
foo   one      foo    one
      two      foo    two

3. Transformando colunas de um dataframe para que se transformem nos índices com hierarquia

In [30]:
df_ = pd.DataFrame({'key1': ['Nevada', 'Ohio', 'Ohio',
                               'Nevada', 'Ohio'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
df_

,key1,key2,data
0,Nevada,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Ohio,2002,4.0


In [35]:
df__=df_.set_index('key1').sort_index()
df__

,key2,data
key1,,
Nevada,2000,0.0
Nevada,2001,3.0
Ohio,2001,1.0
Ohio,2002,2.0
Ohio,2002,4.0


In [27]:
df_.sort_values(by='key1', inplace=True)
df_

,key1,key2,data
0,Nevada,2000,0.0
3,Nevada,2001,3.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
4,Ohio,2002,4.0


In [24]:
df_.set_index('key1')

,key2,data
key1,,
Nevada,2000,0.0
Nevada,2001,3.0
Ohio,2001,1.0
Ohio,2002,2.0
Ohio,2002,4.0


In [39]:
df_.set_index(['key1', 'key2']).sort_index()

data
key1   key2      
Nevada 2000   0.0
       2001   3.0
Ohio   2001   1.0
       2002   2.0
       2002   4.0

Como acessar os dados nesse arranjo de indexação hierarquica?

Se estivermos trabalhando com Series, a indexação irá separar sempre da maior para menor hierarquia. Por exemplo, para a primeira Series que trabalhamos,

In [40]:
data

a  1    0.451539
   2   -0.403930
   3    0.192704
b  1    1.207659
   3   -0.428835
c  1    0.489284
   2   -0.051864
d  2    0.179684
   3   -1.300690
dtype: float64

In [50]:
data['a',3]

0.19270388067170624

In [46]:
data['b':'c']

b  1    1.207659
   3   -0.428835
c  1    0.489284
   2   -0.051864
dtype: float64

In [44]:
data.loc[['b', 'd']]

b  1    1.207659
   3   -0.428835
d  2    0.179684
   3   -1.300690
dtype: float64

In [51]:
data.loc[:, 2]

a   -0.403930
c   -0.051864
d    0.179684
dtype: float64

Quando temos dataframes com indexação hierarquica, a ideia é similar, considerando, obviamente, as formas de acesso as colunas e as linhas. Assim, do nosso primeiro exeplo, temos

In [52]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [53]:
frame['Ohio']

Green  Red
a 1      0    1
  2      3    4
b 1      6    7
  2      9   10

In [54]:
frame['Ohio','Green']

a  1    0
   2    3
b  1    6
   2    9
Name: (Ohio, Green), dtype: int64

In [55]:
frame['Ohio','Green']['a']

1    0
2    3
Name: (Ohio, Green), dtype: int64

In [58]:
frame['Ohio','Green']['a',1]

0

In [59]:
frame.loc['a']

Ohio     Colorado
  Green Red    Green
1     0   1        2
2     3   4        5

In [60]:
frame.loc[('a',1)]

Ohio      Green    0
          Red      1
Colorado  Green    2
Name: (a, 1), dtype: int64

### Reorganizando Níveis em Indexação Hierarquica

Em algumas situações, é necessário que reorganizemos os níveis de indexação, trocando, por exemplo, a ordem dos níveis. Considere, por exemplo, 

In [66]:
frame.index.names = ['Let', 'Num']
frame

Ohio     Colorado
        Green Red    Green
Let Num                   
a   1       0   1        2
    2       3   4        5
b   1       6   7        8
    2       9  10       11

In [67]:
frame.index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           names=['Let', 'Num'])

Se acaso quisessemos que o nível mais externo dos índices das linhas fosse 1 e 2 e não a e b, como poderíamos proceder?

O mais natural seria usar o método `swaplevel`, da seguinte forma:

In [74]:
frame = frame.swaplevel(i='Let', j='Num').sort_index()
frame

Ohio     Colorado
        Green Red    Green
Let Num                   
a   1       0   1        2
    2       3   4        5
b   1       6   7        8
    2       9  10       11

In [73]:
frame.swaplevel(axis=1)

Green  Red    Green
         Ohio Ohio Colorado
Num Let                    
1   a       0    1        2
    b       6    7        8
2   a       3    4        5
    b       9   10       11

O método basicamente realiza a troca entre os níveis que estão referenciados por i e j, que podem receber valores das posições (seguindo a lógica de posições de listas) ou por nomes. no exemplo acima nós trabalhamos por posição, uma vez que os níveis não possuem nome atribuído.

Caso tenhamos somente dois níveis, como nosso caso, os valores _default_ dos parâmetros já fariam o trabalho (veja a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.swaplevel.html))

In [ ]:
frame.swaplevel()

*OBS: lembre-se que essa alteração é feita em uma cópia e não no dataframe original* 

Porém, o resultado que obtemos não ficou muito funcional. Notemos que o primeiro nível não está organizado de forma a mesclar os elementos sob sua hierarquia.  

Quando casos assim ocorrem, é necessário reordenar os índices, usando para isso o método `sort_index`, cuja documentação está disponível [aqui](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_index.html).

Para corrigirmos o problema, precisamos especificar em que nível nós queremos reorganizar, por meio do parâmetro `level`. 

No nosso caso em questão, queremos reordenar no nível 1, mais "interno", para depois poder reorganizar. dessa forma, fazendo

In [75]:
frame.sort_index(level=1)

Ohio     Colorado
        Green Red    Green
Let Num                   
a   1       0   1        2
b   1       6   7        8
a   2       3   4        5
b   2       9  10       11

e reorganizando os níveis,

In [76]:
frame.swaplevel().sort_index()

Ohio     Colorado
        Green Red    Green
Num Let                   
1   a       0   1        2
    b       6   7        8
2   a       3   4        5
    b       9  10       11

## Combinando e mesclando conjuntos de dados

Em muitas situações, nós precisamos juntar, de alguma forma, dois ou mais dataframes para realizar determinadas análises.

Esse, inclusive, é um dos focos do projeto de vocês...

Há duas abordagens básicas quando tentamos juntar dois ou mais dataframes:

1. realizar a concatenação desses dataframes (ou series...) no sentido de um dos eixos (0 se for no sentido de linha, 1 se for de coluna), usando, para isso, o método `concat()`;

2. realizar a mescla (ou fusão, como queira chamar) de dois ou mais dataframes (ou series...) considerando chaves para isso, usando o método `merge()` (essa, abordagem é derivada da abordagem convencional em bancos de dados estruturados).

Vamos explorar cada um desses métodos e suas possibilidades!

### Concatenação de objetos Pandas

Existem duas ações básicas a serem feitas:

![](https://files.realpython.com/media/concat_axis0.2ec65b5f72bc.png)

`pd.concat([df1, df2])`

Vamos considerar os seguintes dataframes

In [77]:
df1 = pd.DataFrame({
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"]},
         index=[0, 1, 2, 3])
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [78]:
df2 = pd.DataFrame({
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"]},
         index=[4, 5, 6, 7])
df2

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7




Assim,

In [87]:
pd.concat([df2,df1], axis=0, ignore_index=True)

,A,B,C,D
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7
4,A0,B0,C0,D0
5,A1,B1,C1,D1
6,A2,B2,C2,D2
7,A3,B3,C3,D3


Note que estamos concatenando no sentido de linha, isto é, ao longo do eixo zero. Isso equivale a "empilhar" os dataframes.

E se quisermos concatenar no sentido do eixo 1, isto é, no sentido de coluna? Basta usarmos o parâmetro _axis=1_, como no exemplo abaixo

![](https://files.realpython.com/media/concat_col.a8eec2b4e84f.png)

`pd.concat([df1, df2], axis=1)`

In [96]:
df3 = pd.DataFrame({
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "G": ["C4", "C5", "C6", "C7"]},
         index=[0,1,2,3])
df3

,A,B,G
0,A4,B4,C4
1,A5,B5,C5
2,A6,B6,C6
3,A7,B7,C7


In [83]:
pd.concat([df3,df1], axis=1)

,E,F,G,A,B,C,D
0,A4,B4,C4,A0,B0,C0,D0
1,A5,B5,C5,A1,B1,C1,D1
2,A6,B6,C6,A2,B2,C2,D2
3,A7,B7,C7,A3,B3,C3,D3


Agora, e se fizermos

In [91]:
pd.concat([df1,df2], axis=1)

,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,NaN,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,A4,B4,C4,D4
5,NaN,NaN,NaN,NaN,A5,B5,C5,D5
6,NaN,NaN,NaN,NaN,A6,B6,C6,D6
7,NaN,NaN,NaN,NaN,A7,B7,C7,D7


In [97]:
pd.concat([df1,df3], axis=0)

,A,B,C,D,G
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
0,A4,B4,NaN,NaN,C4
1,A5,B5,NaN,NaN,C5
2,A6,B6,NaN,NaN,C6
3,A7,B7,NaN,NaN,C7


Por que esse monte de _nan_?

Note a presená de um número significativo de _nan_. Lembre-se que _nan_ em Pandas significa dado faltoso. E por que faltoso? Por que nenhuma das coluna tem valores em todos os índices, e eles são mantidos!!

Nós podemos criar também indexação hierarquica na criação a partir do concat.

In [98]:
pd.concat([df1,df2], keys=["Um", "Dois"])

A   B   C   D
Um   0  A0  B0  C0  D0
     1  A1  B1  C1  D1
     2  A2  B2  C2  D2
     3  A3  B3  C3  D3
Dois 4  A4  B4  C4  D4
     5  A5  B5  C5  D5
     6  A6  B6  C6  D6
     7  A7  B7  C7  D7

In [99]:
pd.concat([df1,df3], axis=1, keys=["Um", "Dois"])

Um             Dois        
    A   B   C   D    A   B   G
0  A0  B0  C0  D0   A4  B4  C4
1  A1  B1  C1  D1   A5  B5  C5
2  A2  B2  C2  D2   A6  B6  C6
3  A3  B3  C3  D3   A7  B7  C7

### Combinando dataframes no estilo de banco de dados

`merge()` é um método que traz para os dataframes Pandas a possibilidade de se combinar do mesmo modo que bancos de dados relacionais fazem: por meio de uma ou mais chaves.



In [100]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [112]:
df2 = pd.DataFrame({'key': ['a', 'b', 'b'],
                    'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,b,2


In [113]:
df3 = df2

Junção do tipo _muitos-para-um_:

In [116]:
pd.merge(df3, df2)

,key,data2
0,a,0
1,b,1
2,b,2


*OBS:* A saída dos valores __sempre__ será um produto cartesiano entre os valores associados a cada chave nos dois conjuntos de dados!

Boa prática: especificar a coluna para fazer a junção (caso não o faça, como no exemplo anterior, o pandas considera a coluna que se sobrepõe)

In [117]:
pd.merge(df3, df2, on='key')

,key,data2_x,data2_y
0,a,0,0
1,b,1,1
2,b,1,2
3,b,2,1
4,b,2,2


Questões surgidas...

1. na coluna `keys`, não se consideraram os valores _c_ de df1 e _d_ de df2. Por quê?
    * padrão `inner` - intersecção
    * para mudar, basta usar o parametro `how`, com três opções:
        - `outer`: usa a união de todas as chaves das tabelas combinadas
        - `left`: usa as combinações de chaves da tabela à esquerda
        - `right`: usa as combinações de chaves da tabela à direita

In [104]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [105]:
pd.merge(df1, df2, how='left')

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [106]:
pd.merge(df1, df2, how='right')

,key,data1,data2
0,a,2.0,0
1,a,4.0,0
2,a,5.0,0
3,b,0.0,1
4,b,1.0,1
5,b,6.0,1
6,d,NaN,2


### Merge no index

In [148]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': ['0', '1', '2', '3', '4', '5']})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [153]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1.index.name = 'key'

right1

,group_val
key,
a,3.5
b,7.0


In [154]:
right1.reset_index(inplace=True)
right1

,key,group_val
0,a,3.5
1,b,7.0


In [133]:
right1.index

Index(['a', 'b'], dtype='object', name='key')

In [155]:
pd.merge(left1, right1, on='key')

,key,value,group_val
0,a,0,3.5
1,a,2,3.5
2,a,3,3.5
3,b,1,7.0
4,b,4,7.0


In [127]:
pd.merge(left1, right1, on='key', right_index=True)

MergeError: Can only pass argument "on" OR "left_index" and "right_index", not a combination of both.

O índice da linha da tabela direita (determinada por `right_index` ) é usada como chave para a junção com a coluna da esquerda (determinada por `left_on`. Novamente, `inner` é o padrão.

Para sair desse padrão e fazer a união,

In [147]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


Se os dados forem hierarquicamente indexados,

In [135]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [136]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [137]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [138]:
A = pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')
A

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [139]:
dfh = A.set_index(['key1','key2'])
dfh

data  event1  event2
key1   key2                      
Ohio   2000   0.0     4.0     5.0
       2000   0.0     6.0     7.0
       2001   1.0     8.0     9.0
       2002   2.0    10.0    11.0
Nevada 2001   3.0     0.0     1.0
       2002   4.0     NaN     NaN
       2000   NaN     2.0     3.0

Para uma junção mais fácil pelo índice, pode-se usar, alternativamente, o método `join`.

In [140]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [141]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [142]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [143]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN
